In [10]:
import torch
from datasets.car_evaluation import CarEvaluation
from binhd.embeddings import CategoricalEncoder
from binhd.datasets import BaseDataset
from torch.utils.data import DataLoader
from module.record_encoder import RecordEncoder
from binhd.classifiers import BinHD
import torchmetrics

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [12]:
dataset = CarEvaluation()
dimension = 1000
num_levels = 100
batch_size = 100

In [13]:
X = dataset.features[dataset.categorical_features]

categorical_encoder = CategoricalEncoder(dimension)
X = categorical_encoder.fit_transform(X)

In [14]:
from sklearn.model_selection import train_test_split


y = list(dataset.labels)
y = torch.tensor(y).squeeze().to(device)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)  

train_dataset = BaseDataset(X_train.values, y_train)
test_dataset = BaseDataset(X_test.values, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [15]:
record_encode = RecordEncoder(dimension, dataset.num_features, num_levels)
record_encode = record_encode.to(device)

In [16]:
model = BinHD(dimension, dataset.num_classes)

In [17]:
with torch.no_grad():
    for samples, labels in train_loader:
        samples = samples.to(device)
        labels = labels.to(device)
        samples_cat_hv = categorical_encoder(samples[:,:categorical_encoder.num_features])
        samples_hv = record_encode(samples_cat_hv, samples[:, categorical_encoder.num_features:])
        model.fit(samples_hv, labels)

In [18]:
accuracy = torchmetrics.Accuracy("multiclass", num_classes=dataset.num_classes)    

for samples, labels in test_loader:
    samples = samples.to(device)
    labels = labels.to(device)    
    samples_cat_hv = categorical_encoder(samples[:,:categorical_encoder.num_features])
    samples_hv = record_encode(samples_cat_hv, samples[:, categorical_encoder.num_features:])        
    predictions = model.predict(samples_hv)  
    accuracy.update(predictions, labels)

acc = accuracy.compute().item()    
print("BinHD: Accuracy = ", acc)

BinHD: Accuracy =  0.17533718049526215
